# Lecture1 Talk to LLM

「データサイエンス特論2024」AIIT、岩政より。

- このノートブックは、rinna社によるnekomataという大規模言語モデル(LLM)を使ったQ&Aの実行例です
- このノートブックは、Google Colabで、こんなことまでできるという最新の事例を紹介するために用意しました。(岩政)

-
- 最初に必要なライブラリをgoogle colab環境にインストールします。

In [1]:
!pip install tiktoken
!pip install einops transformers_stream_generator
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 764.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for transformers_stream_generator: filename=transformers_stream_generator-0.0.5-py3-none-any.whl size=12425 sha256=898c3f5dc29b798982d76af2d2e88f6bbb4648dc4dff0ae5cd005c2988beee6b
  Stored in directory: /root/.cache/pip/wheels/95/4a/90/140f7b67d125906f6a165f38aad212ecb4a695ad0d87582437
Successfully built transformers_stream_generator
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 89.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

このnotebookでは、[huggingface](https://huggingface.co/)という、機械学習のモデルを提供するサービスから、[rinna社](https://rinna.co.jp/)が提供する nekomata-7b-instractionをダウンロードしてモデル(model)として使います。

モデルの実体は、https://huggingface.co/rinna/nekomata-7b-instruction　にあります。詳しい仕様と必要なファイル群があります。


nekomata-7b-instractionは、アリババ社が公開したQwenシリーズ（日本語がお得意とされている）に、rinna社が日本語の学習データを用いて継続事前学習を行って公開した、「Nekomataシリーズ」から、対話形式(instruct)でユーザーの指示に応答するモデルを選びました。



### Step1:モデルのダウンロード
- PyTorchという機械学習用のライブラリ使います (torch)
- 「Huggingface Transformers」という、深層学習向けのライブラリを使います(transformers)

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("rinna/nekomata-7b-instruction", trust_remote_code=True)

# Automatically select device and precision
model = AutoModelForCausalLM.from_pretrained("rinna/nekomata-7b-instruction", device_map="auto", trust_remote_code=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

tokenization_qwen.py:   0%|          | 0.00/9.62k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/rinna/nekomata-7b-instruction:
- tokenization_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


qwen.tiktoken:   0%|          | 0.00/2.56M [00:00<?, ?B/s]

...

cpp_kernels.py:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/rinna/nekomata-7b-instruction:
- cpp_kernels.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/rinna/nekomata-7b-instruction:
- modeling_qwen.py
- qwen_generation_utils.py
- cpp_kernels.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/15.4G [00:00<?, ?B/s]


以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示:
次の日本語を英語に翻訳してください。

### 入力:
大規模言語モデル（だいきぼげんごモデル、英: large language model、LLM）は、多数のパラメータ（数千万から数十億）を持つ人工ニューラルネットワークで構成されるコンピュータ言語モデルで、膨大なラベルなしテキストを使用して自己教師あり学習または半教師あり学習によって訓練が行われる。

### 応答:
A large language model (LLM) is a computer language model consisting of an artificial neural network with millions or billions of parameters that is trained using self-supervised or semi-supervised learning using vast amounts of unlabeled text.<|endoftext|>


'\n以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n次の日本語を英語に翻訳してください。\n\n### 入力:\n大規模言語モデル（だいきぼげんごモデル、英: large language model、LLM）は、多数のパラメータ（数千万から数十億）を持つ人工ニューラルネットワークで構成されるコンピュータ言語モデルで、膨大なラベルなしテキストを使 用して自己教師あり学習または半教師あり学習によって訓練が行われる。\n\n### 応答:\n A large language model (LLM) is a computer language model composed of artificial neural networks with many parameters (from tens of millions to billions) trained by self-supervised learning or semi-supervised learning using a large amount of unlabeled text.<|endoftext|>\n'

- promptというのが、LLMに対する支持です
- input変数に与える文字列をに対する答えを返します
- あとはLLMを使うための定型処理なので興味のある方は調べてみてください。
 - generate関数でpromptに対する答えを得ている
 - tokenizer.encodeで、promptをLLMで読めるよう符号化します
 - temperature(0..1)は出力のランダム性の指定で、引くほどランダム性が減る
 - 最後にLLMの出力は符号なので、これを自然言語にdecodeしています。

In [6]:
# プロンプトの準備
instruction= "あなたはQAアシスタントです。ユーザーの質問に対して回答とその理由を返答してください。"
input = "江戸時代は、どうして約２００年も続いたのか？"
input = "風の谷のナウシカにおける、ナウシカと森の人との関係性について"

prompt = f"""
以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示:
{instruction}

### 入力:
{input}

### 応答:
"""

# 推論の実行
with torch.no_grad():
    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
        repetition_penalty=1.0,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
output = tokenizer.decode(output_ids[0][token_ids.size(1) :])
print(output)

ナウシカは、森の生きているものの声を聞くことができ、森を守護する力を持つ「森の人」と見なされています。ナウシカが「森の人」として認められ、その力を発揮しなければならない場面はたくさんあります。<|endoftext|>
